## INK: explanatory notebook

Within this notebook, we give a simple example of how the INK library can be used to extract the neighbourhood of certain subjects of interest.<br>
We also show the mining capabilities for both task specific and task agnostic cases.


The example dataset can be found in the /datasets folder.<br>
We have used the common known animal dataset, describing several animals with their properties.

In [1]:
from ink.base.connectors import RDFLibConnector
from ink.base.structure import InkExtractor
from ink.miner.rulemining import RuleSetMiner

To start, three different packages are loaded.
* A connector: which is used to load the original dataset.
Here we have used an RDFLib connector, but other connectors are available.
* The INK extractor, which will transform the neighbourhood of certain nodes into a binary representation.
* The Rule Set Miner, which is our rule mining module.

### 1. create connector

In [2]:
connector = RDFLibConnector('ink/datasets/animals.owl', 'xml')

The RDLib connector simply requires the filename of the KG to be loaded and the accompanied formatting file.<br>
Querying will be the most important function within such a connector. SPARQL queries can be executed as follows:

In [3]:
connector.query("Select ?s where {?s a <http://dl-learner.org/benchmark/dataset/animals/T-Rex>.}")

[{'s': {'type': 'uri',
   'value': 'http://dl-learner.org/benchmark/dataset/animals#trex01'}}]

### 2. create the extractor
INK extractor takes a connector as argument. We also provide a list of prefixes.

In [4]:
prefix={
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf#",
    "http://www.w3.org/2000/01/rdf-schema#": "rdfs#",
    "http://www.w3.org/2002/07/owl#": "owl#",
    "http://dl-learner.org/benchmark/dataset/animals/": "animals/"
}
extractor = InkExtractor(connector, prefixes=prefix)

In [5]:
extractor

### 3. create the rule miner
All arguments are optinal here, take a look at the documentation for morre information and how they influence the mining capabilities.

In [6]:
miner = RuleSetMiner(chains=100,max_len_rule_set=3, forest_size=10)

## <ins>Task specific rule mining</ins>
We define two different sets of animals:
* one containing only mammals
* non mammals in the other set

In [7]:
pos = set([ "http://dl-learner.org/benchmark/dataset/animals#dog01",
            "http://dl-learner.org/benchmark/dataset/animals#dolphin01",
            "http://dl-learner.org/benchmark/dataset/animals#platypus01",
            "http://dl-learner.org/benchmark/dataset/animals#bat01"])

neg = set(["http://dl-learner.org/benchmark/dataset/animals#trout01",
            "http://dl-learner.org/benchmark/dataset/animals#herring01",
            "http://dl-learner.org/benchmark/dataset/animals#shark01",
            "http://dl-learner.org/benchmark/dataset/animals#lizard01",
            "http://dl-learner.org/benchmark/dataset/animals#croco01",
            "http://dl-learner.org/benchmark/dataset/animals#trex01",
            "http://dl-learner.org/benchmark/dataset/animals#turtle01",
            "http://dl-learner.org/benchmark/dataset/animals#eagle01",
            "http://dl-learner.org/benchmark/dataset/animals#ostrich01",
            "http://dl-learner.org/benchmark/dataset/animals#penguin01"])

In [8]:
pos

{'http://dl-learner.org/benchmark/dataset/animals#bat01',
 'http://dl-learner.org/benchmark/dataset/animals#dog01',
 'http://dl-learner.org/benchmark/dataset/animals#dolphin01',
 'http://dl-learner.org/benchmark/dataset/animals#platypus01'}

The goal for ink is to learn a rule to seperate the pos set (mammals) from the negative set (non mammals).
First: we extract the neighbourhoods until a certain depth of both the nodes in the pos set and neg set.

In [9]:
X_train, y_train = extractor.create_dataset(4, pos, neg, jobs=4)

In [10]:
X_train

[('http://dl-learner.org/benchmark/dataset/animals#platypus01',
  {'rdf#type': ['owl#NamedIndividual', 'animals/Platypus'],
   'rdf#type.rdf#type': ['owl#Class'],
   'rdf#type.rdfs#subClassOf': ['animals/Animal',
    'animals/HasEggs',
    'animals/HasMilk',
    'animals/Homeothermic',
    'N404e2526b5024dfdbe106b1ead7a8c02',
    'N1846aaccbeec422ab8636bd9255df4c9',
    'N954c5036ead844ee89ac707e83b1b9a6'],
   'rdf#type.rdfs#subClassOf.rdf#type': ['owl#Class',
    'owl#Class',
    'owl#Class',
    'owl#Class'],
   'rdf#type.rdfs#subClassOf.rdfs#subClassOf': ['animals/Animal',
    'animals/Animal',
    'animals/Animal'],
   'rdf#type.rdfs#subClassOf.rdfs#subClassOf.rdf#type': ['owl#Class',
    'owl#Class',
    'owl#Class']}),
 ('http://dl-learner.org/benchmark/dataset/animals#shark01',
  {'rdf#type': ['owl#NamedIndividual', 'animals/Shark'],
   'rdf#type.rdf#type': ['owl#Class'],
   'rdf#type.rdfs#subClassOf': ['animals/Animal',
    'animals/HasEggs',
    'animals/HasGills',
    'Nd3452

X_train stores the neighbourhood of each node inside the pos set and neg set as a dictionary of (predicate,object) values.<br>
An example animal is given below:

In [11]:
type(X_train)

list

In [12]:
print("animal:", X_train[0][0])
for x in X_train[0][1]:
    print("relation:",x)
    print("objects:",X_train[0][1][x])

animal: http://dl-learner.org/benchmark/dataset/animals#platypus01
relation: rdf#type
objects: ['owl#NamedIndividual', 'animals/Platypus']
relation: rdf#type.rdf#type
objects: ['owl#Class']
relation: rdf#type.rdfs#subClassOf
objects: ['animals/Animal', 'animals/HasEggs', 'animals/HasMilk', 'animals/Homeothermic', 'N404e2526b5024dfdbe106b1ead7a8c02', 'N1846aaccbeec422ab8636bd9255df4c9', 'N954c5036ead844ee89ac707e83b1b9a6']
relation: rdf#type.rdfs#subClassOf.rdf#type
objects: ['owl#Class', 'owl#Class', 'owl#Class', 'owl#Class']
relation: rdf#type.rdfs#subClassOf.rdfs#subClassOf
objects: ['animals/Animal', 'animals/Animal', 'animals/Animal']
relation: rdf#type.rdfs#subClassOf.rdfs#subClassOf.rdf#type
objects: ['owl#Class', 'owl#Class', 'owl#Class']


y_train contains the class vector indicating to which group each animal belongs.

In [13]:
y_train

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0])

Next, we contruct from these dictionaries the binary INK representation


In [14]:
X_train = extractor.fit_transform(X_train, counts=True, levels=True)

The counts and levels modules were enabled. These modules are responsible for 1) counting the number of available objects related to a subject and 2) add additional values for > and < relations when all objects are numbers.
<br>
The transformed X_train variable is a tuple consisting of 3 parts:
* a sparse boolean matrix
* a list of indices (here the uri's to the animals)
* a list of column values

This X_train tuple can be transformed into a pandas dataframe:

In [15]:
X_train

(<14x164 sparse matrix of type '<class 'numpy.bool_'>'
 	with 992 stored elements in Compressed Sparse Row format>,
 ['http://dl-learner.org/benchmark/dataset/animals#platypus01',
  'http://dl-learner.org/benchmark/dataset/animals#shark01',
  'http://dl-learner.org/benchmark/dataset/animals#dog01',
  'http://dl-learner.org/benchmark/dataset/animals#herring01',
  'http://dl-learner.org/benchmark/dataset/animals#trex01',
  'http://dl-learner.org/benchmark/dataset/animals#ostrich01',
  'http://dl-learner.org/benchmark/dataset/animals#penguin01',
  'http://dl-learner.org/benchmark/dataset/animals#croco01',
  'http://dl-learner.org/benchmark/dataset/animals#eagle01',
  'http://dl-learner.org/benchmark/dataset/animals#dolphin01',
  'http://dl-learner.org/benchmark/dataset/animals#trout01',
  'http://dl-learner.org/benchmark/dataset/animals#bat01',
  'http://dl-learner.org/benchmark/dataset/animals#turtle01',
  'http://dl-learner.org/benchmark/dataset/animals#lizard01'],
 ['count.rdf#type',
 

In [16]:
X_train[0]

<14x164 sparse matrix of type '<class 'numpy.bool_'>'
	with 992 stored elements in Compressed Sparse Row format>

In [17]:
type(X_train)

tuple

In [19]:
import pandas as pd
df_train = pd.DataFrame.sparse.from_spmatrix(X_train[0], index=X_train[1], columns=X_train[2])
df_train.head()

,count.rdf#type,count.rdf#type.rdfs#subClassOf,count.rdf#type.rdfs#subClassOf.rdf#type,count.rdf#type.rdfs#subClassOf.rdf#type.owl#Class,count.rdf#type.rdfs#subClassOf.rdf#type.owl#Class<=2,count.rdf#type.rdfs#subClassOf.rdf#type.owl#Class<=3,count.rdf#type.rdfs#subClassOf.rdf#type.owl#Class<=4,count.rdf#type.rdfs#subClassOf.rdf#type.owl#Class>=2,count.rdf#type.rdfs#subClassOf.rdf#type.owl#Class>=3,count.rdf#type.rdfs#subClassOf.rdf#type.owl#Class>=4,...,rdf#type§animals/Herring,rdf#type§animals/Lizard,rdf#type§animals/Ostrich,rdf#type§animals/Penguin,rdf#type§animals/Platypus,rdf#type§animals/Shark,rdf#type§animals/T-Rex,rdf#type§animals/Trout,rdf#type§animals/Turtle,rdf#type§owl#NamedIndividual
http://dl-learner.org/benchmark/dataset/animals#platypus01,True,True,True,True,False,False,True,True,True,True,...,0,0,0,0,1,0,0,0,0,True
http://dl-learner.org/benchmark/dataset/animals#bat01,True,True,True,True,False,True,True,True,True,False,...,0,0,0,0,0,0,0,0,0,True
http://dl-learner.org/benchmark/dataset/animals#trex01,True,True,True,True,True,True,True,True,False,False,...,0,0,0,0,0,0,1,0,0,True
http://dl-learner.org/benchmark/dataset/animals#trout01,True,True,True,True,False,True,True,True,True,False,...,0,0,0,0,0,0,0,1,0,True
http://dl-learner.org/benchmark/dataset/animals#ostrich01,True,True,True,True,False,True,True,True,True,False,...,0,0,1,0,0,0,0,0,0,True


To mine rules, we can simple provide the X_train tuple to the rule mining module. <br>
Together with the labels (y_train), this will rule miner will now try to find a rule to descriminate the pos from the neg set.



In [19]:
acc, rules = miner.fit(X_train, y_train)

C:\Users\cherr\OneDrive\Desktop\2266\INK\ink\miner\rulemining.py:223: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.rules = np.asarray(self.rules)[supp_select]


The output of this miner is both the achieved accuracy and the found rules. <br>
We can print the rules as follows:

In [20]:
print(acc)
miner.print_rules(rules)

1.0
['rdf#type.rdfs#subClassOf§animals/HasMilk']


with 100% accuracy, we can separate the positive class of rules from the negative ones by using the rule above.
This rule state that starting from the individual of a certain type ?x which has a subclass ?y with the relation hasMilk is a mammal.



## <ins>Task agnostic rule mining</ins>

A similar setup can be used to mine task agnostic rules. 
<br>The only difference is that no training labels need to be provided.

We no define a general SPARQL query to select our nodes of interest:

In [14]:
query = """
SELECT ?s WHERE {
    ?s a ?o.
    ?o rdfs:subClassOf <http://dl-learner.org/benchmark/dataset/animals/Animal>.
}
"""

This query can be used directly in our neighbourhoud extractor.

In [15]:
X_train, _ = extractor.create_dataset(10, query, None)

As you can see, we provide no negative set and do not store the y_train values (because they are all positive). <br>
Next, we transform these neighbourhoods into the INK binary representation.

In [16]:
X_train = extractor.fit_transform(X_train)

X_train can now be used to mine task-agnostic rules. We created

In [17]:
miner.support = 10
miner.fit(X_train)

antecedents  \
0                (rdf#type.rdfs#subClassOf.rdf#type)   
1                                (rdf#type.rdf#type)   
2  (rdf#type.rdfs#subClassOf.rdfs#subClassOf.rdf#...   
3                                (rdf#type.rdf#type)   
4                (rdf#type.rdfs#subClassOf.rdf#type)   
5  (rdf#type.rdfs#subClassOf.rdfs#subClassOf.rdf#...   
6         (rdf#type.rdfs#subClassOf.rdfs#subClassOf)   
7                         (rdf#type.rdfs#subClassOf)   

                                         consequents  antecedent support  \
0                                (rdf#type.rdf#type)                20.0   
1                (rdf#type.rdfs#subClassOf.rdf#type)                20.0   
2                                (rdf#type.rdf#type)                17.0   
3  (rdf#type.rdfs#subClassOf.rdfs#subClassOf.rdf#...                20.0   
4  (rdf#type.rdfs#subClassOf.rdfs#subClassOf.rdf#...                20.0   
5                (rdf#type.rdfs#subClassOf.rdf#type)                17.0   
6                         (rdf#type.rdfs#subClassOf)                22.0   
7         (rdf#type.rdfs#subClassOf.rdfs#subClassOf)                99.0   

   consequent support  support  confidence      lift  leverage  conviction  
0                20.0     20.0    1.000000  0.050000    -380.0         inf  
1                20.0     20.0    1.000000  0.050000    -380.0         inf  
2                20.0     17.0    1.000000  0.050000    -323.0         inf  
3                17.0     17.0    0.850000  0.050000    -323.0 -106.666667  
4                17.0     17.0    0.850000  0.050000    -323.0 -106.666667  
5                20.0     17.0    1.000000  0.050000    -323.0         inf  
6                99.0     22.0    1.000000  0.010101   -2156.0         inf  
7                22.0     22.0    0.222222  0.010101   -2156.0  -27.000000

The resulting table shows all mined rules for this simple example. <br>
The confidence, lift and leverage measures can be used to further evaluate these mined rules.